# Эксперимент №7(соревнование kaggle)
https://www.kaggle.com/competitions/physical-activity-clustering

In [33]:
import sys
import os

sys.path.append('..')
folder_path = os.path.abspath(os.path.join(os.getcwd(), "../../evaluate"))
sys.path.append(folder_path)

In [34]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tools import deletion_features
from tools import balancing
from tools import encoder
from tools import scaling
from Alg_CatBoost import CatBoostRegressor_fill 
from Alg_DecisionTree import DecisionTreeRegressor_fill
from Alg_Interpolate import interpolate_fill
from Alg_KNNImputer import KNNImputer_fill
from Alg_LinearRegression import LinearRegression_fill
from Alg_max import max_fill
from Alg_mean import mean_fill 
from Alg_median import median_fill
from Alg_min import min_fill
from Alg_RandomForestRegressor import RandomForestRegressor_fill
from Alg_SVR import SVR_fill
from Alg_XGBRegressor import XGBRegressor_fill
import results_metrics
import results_metrics_visualization

In [35]:
df = pd.read_csv('./Data/Physical_Activity_Monitoring_unlabeled.csv')

In [36]:
df.shape

(534601, 53)

# 1. Исследуем данные

In [37]:
df.head()

,timestamp,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,handGyro2,...,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,ankleOrientation1,ankleOrientation2,ankleOrientation3,ankleOrientation4,subject_id
0,252.75,34.8750,0.902847,7.45872,6.032490,1.006100,7.44575,6.303650,-0.021952,0.004114,...,-0.025606,0.030239,-14.5447,27.3336,-5.76620,0.206416,0.348212,0.763201,-0.503652,5
1,3652.45,29.7500,-4.069330,3.76168,8.279720,-5.972500,3.22273,6.908970,0.116054,-0.375968,...,-0.670163,1.786200,-32.8257,-15.7719,-1.31395,0.578332,-0.566734,0.405888,-0.423791,2
2,504.76,34.3750,-2.007540,-9.60180,0.773674,-1.803500,-9.58937,1.073680,0.020131,0.042189,...,-0.009741,-0.006579,-18.8171,16.3089,25.29280,0.139776,0.676586,0.393796,0.606316,8
3,2801.61,31.9375,-13.262200,5.85443,0.899334,-13.611500,5.91511,0.840377,1.715630,-0.050492,...,-0.163470,-3.495570,-26.7418,34.3566,21.32740,0.416303,0.783387,0.306464,0.345075,2
4,441.01,33.5000,-0.013940,8.66540,4.398210,0.144153,8.56816,4.822200,-0.006938,-0.007383,...,-0.028172,-0.003737,-17.7798,-3.9981,10.99500,0.171133,-0.748343,0.252988,-0.588807,6


In [38]:
#проверим данные на типы
df.dtypes

timestamp            float64
handTemperature      float64
handAcc16_1          float64
handAcc16_2          float64
handAcc16_3          float64
handAcc6_1           float64
handAcc6_2           float64
handAcc6_3           float64
handGyro1            float64
handGyro2            float64
handGyro3            float64
handMagne1           float64
handMagne2           float64
handMagne3           float64
handOrientation1     float64
handOrientation2     float64
handOrientation3     float64
handOrientation4     float64
chestTemperature     float64
chestAcc16_1         float64
chestAcc16_2         float64
chestAcc16_3         float64
chestAcc6_1          float64
chestAcc6_2          float64
chestAcc6_3          float64
chestGyro1           float64
chestGyro2           float64
chestGyro3           float64
chestMagne1          float64
chestMagne2          float64
chestMagne3          float64
chestOrientation1    float64
chestOrientation2    float64
chestOrientation3    float64
chestOrientati

In [39]:
#проверим на пропуски
df.isna().sum()

timestamp               0
handTemperature      4041
handAcc16_1          4041
handAcc16_2          4041
handAcc16_3          4041
handAcc6_1           4041
handAcc6_2           4041
handAcc6_3           4041
handGyro1            4041
handGyro2            4041
handGyro3            4041
handMagne1           4041
handMagne2           4041
handMagne3           4041
handOrientation1     4041
handOrientation2     4041
handOrientation3     4041
handOrientation4     4041
chestTemperature      809
chestAcc16_1          809
chestAcc16_2          809
chestAcc16_3          809
chestAcc6_1           809
chestAcc6_2           809
chestAcc6_3           809
chestGyro1            809
chestGyro2            809
chestGyro3            809
chestMagne1           809
chestMagne2           809
chestMagne3           809
chestOrientation1     809
chestOrientation2     809
chestOrientation3     809
chestOrientation4     809
ankleTemperature     2546
ankleAcc16_1         2546
ankleAcc16_2         2546
ankleAcc16_3

In [40]:
df.columns

Index(['timestamp', 'handTemperature', 'handAcc16_1', 'handAcc16_2',
       'handAcc16_3', 'handAcc6_1', 'handAcc6_2', 'handAcc6_3', 'handGyro1',
       'handGyro2', 'handGyro3', 'handMagne1', 'handMagne2', 'handMagne3',
       'handOrientation1', 'handOrientation2', 'handOrientation3',
       'handOrientation4', 'chestTemperature', 'chestAcc16_1', 'chestAcc16_2',
       'chestAcc16_3', 'chestAcc6_1', 'chestAcc6_2', 'chestAcc6_3',
       'chestGyro1', 'chestGyro2', 'chestGyro3', 'chestMagne1', 'chestMagne2',
       'chestMagne3', 'chestOrientation1', 'chestOrientation2',
       'chestOrientation3', 'chestOrientation4', 'ankleTemperature',
       'ankleAcc16_1', 'ankleAcc16_2', 'ankleAcc16_3', 'ankleAcc6_1',
       'ankleAcc6_2', 'ankleAcc6_3', 'ankleGyro1', 'ankleGyro2', 'ankleGyro3',
       'ankleMagne1', 'ankleMagne2', 'ankleMagne3', 'ankleOrientation1',
       'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4',
       'subject_id'],
      dtype='object')

In [41]:
print(len(df.columns))

53


# 2.3. Устранение дубликатов

In [42]:
df.shape

(534601, 53)

In [43]:
#проверяем наличие дубликатов и удаляем их
data = df.drop_duplicates()
data.shape

(534601, 53)

Проверяем на аномалии и устраняем их

Информация об атрибутах:
- LB - базовый уровень ЧСС (ударов в минуту)
- AC - количество ускорений в секунду
- FM - количество движений плода в секунду
- UC - количество сокращений матки в секунду
- DL - количество легких замедлений в секунду
- DS - количество серьезных замедлений в секунду
- DP - # длительных замедлений в секунду
- ASTV - процент времени с аномальной краткосрочной изменчивостью
- MSTV - среднее значение краткосрочной изменчивости
- ALTV - процент времени с аномальной долговременной изменчивостью
- MLTV - среднее значение долговременной изменчивости
- Width - ширина гистограммы FHR
- Min - минимум гистограммы FHR
- Max — максимум гистограммы FHR
- Nmax — количество пиков гистограммы
- Nzeros — количество нулей гистограммы
- Mode — режим гистограммы
- Mean — среднее значение гистограммы
- Median — медиана гистограммы
- Variance — дисперсия гистограммы
- Tendency — тенденция гистограммы
- CLASS — код класса паттерна FHR (от 1 до 10)
- NSP — код класса состояния плода (N = нормальное; S = подозрительное; P = патологическое)

In [44]:
col_name = data.columns
col_name

Index(['timestamp', 'handTemperature', 'handAcc16_1', 'handAcc16_2',
       'handAcc16_3', 'handAcc6_1', 'handAcc6_2', 'handAcc6_3', 'handGyro1',
       'handGyro2', 'handGyro3', 'handMagne1', 'handMagne2', 'handMagne3',
       'handOrientation1', 'handOrientation2', 'handOrientation3',
       'handOrientation4', 'chestTemperature', 'chestAcc16_1', 'chestAcc16_2',
       'chestAcc16_3', 'chestAcc6_1', 'chestAcc6_2', 'chestAcc6_3',
       'chestGyro1', 'chestGyro2', 'chestGyro3', 'chestMagne1', 'chestMagne2',
       'chestMagne3', 'chestOrientation1', 'chestOrientation2',
       'chestOrientation3', 'chestOrientation4', 'ankleTemperature',
       'ankleAcc16_1', 'ankleAcc16_2', 'ankleAcc16_3', 'ankleAcc6_1',
       'ankleAcc6_2', 'ankleAcc6_3', 'ankleGyro1', 'ankleGyro2', 'ankleGyro3',
       'ankleMagne1', 'ankleMagne2', 'ankleMagne3', 'ankleOrientation1',
       'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4',
       'subject_id'],
      dtype='object')

# 2.5. Масштабирование данных

Произведем масштабирование данных

In [45]:
df1 = scaling.data_scaling(data)

In [46]:
df1.shape

(534601, 53)

# 3. Работаем с пропусками

data - после предобработки данных был получен эталонный датасет с которым будет происходить сравнение по метрикам

# 3.2. Заполняем пропуски

#1. Заполним пропуски средним арифметическим значением по признаку

In [21]:
df_mean_fill = mean_fill.mean_fill(df1)

#2. Заполним пропуски медианным значением по признаку

In [22]:
df_median_fill = median_fill.median_fill(df1)

#3. Заполним пропуски минимальным значением по признаку

In [23]:
df_min_fill = min_fill.min_fill(df1)

#4. Заполним пропуски максимальным значением по признаку

In [24]:
df_max_fill = df_min_fill = max_fill.max_fill(df1)

#5. Заполним пропуски используя интерполяцию

In [25]:
df_interpolate_fill = interpolate_fill.interpolate_fill(df1)

In [47]:
df = df1

In [49]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import random

random.seed(42)

col = df.loc[:, df.isna().sum() > 0].columns[0]

# обучающей выборкой будут строки без пропусков
train = df.dropna().copy()
# тестовой (или вернее выборкой для заполнения пропусков)
# будут те строки, в которых пропуски есть
test = df[df[col].isna()].copy()

# выясним индекс столбца с пропусками
col_index = df.columns.get_loc(col)

# разделим на features и target
y_train = train[col]
X_train = train.drop(col, axis=1)

# из "тестовой" выборки удалим столбец с пропусками
test = test.drop(col, axis=1)

# обучим модель линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [50]:
col

'handTemperature'

#6. Заполним пропуски используя линейную регрессию

In [ ]:
df_LinearReg_fill = LinearRegression_fill.linreg_imputer(df1)

#7 Заполним пропуски используя KNNImputer

In [ ]:
df_KNNImputer_fill = KNNImputer_fill.KNNImputer_fill(df1)

#8 Заполним пропуски используя дерево решений

In [ ]:
df_DecisionTree_fill = DecisionTreeRegressor_fill.DecisionTree_imputer(df1)

#9 Заполним пропуски используя случайный лес

In [ ]:
df_RandomForest_fill = RandomForestRegressor_fill.RandomForest_imputer(df1)

#10 Заполним пропуски используя SVM

In [ ]:
df_SVM_fill = SVR_fill.SVM_imputer(df1)

#11 Заполним пропуски используя градиентный бустинг - XGboost

In [ ]:
df_XGBRegressor_fill = XGBRegressor_fill.XGBRegressor_imputer(df1)

#12 Заполним пропуски используя градиентный бустинг - CatBoost

In [ ]:
df_CatBoostRegressor_fill = CatBoostRegressor_fill.CatBoostRegressor_imputer(df1)

# 4. Формируем результаты эксперимента

In [ ]:
all_methods = [df_mean_fill, df_median_fill, df_min_fill, df_max_fill, df_interpolate_fill, df_LinearReg_fill, df_KNNImputer_fill, df_DecisionTree_fill, df_RandomForest_fill, df_SVM_fill, df_XGBRegressor_fill, df_CatBoostRegressor_fill]
df1_result = results_metrics.generate_results_metrics(data, all_methods)
df1_result

In [ ]:
all_methods = [mean_fill.mean_fill(df15), median_fill.median_fill(df15), min_fill.min_fill(df15), max_fill.max_fill(df15), interpolate_fill.interpolate_fill(df15),
              LinearRegression_fill.linreg_imputer(df15), KNNImputer_fill.KNNImputer_fill(df15), DecisionTreeRegressor_fill.DecisionTree_imputer(df15),
              RandomForestRegressor_fill.RandomForest_imputer(df15), SVR_fill.SVM_imputer(df15), XGBRegressor_fill.XGBRegressor_imputer(df15), CatBoostRegressor_fill.CatBoostRegressor_imputer(df15)]
df15_result = results_metrics.generate_results_metrics(data, all_methods)
df15_result

In [ ]:
all_methods = [mean_fill.mean_fill(df20), median_fill.median_fill(df20), min_fill.min_fill(df20), max_fill.max_fill(df20), interpolate_fill.interpolate_fill(df20), 
              LinearRegression_fill.linreg_imputer(df20), KNNImputer_fill.KNNImputer_fill(df20), DecisionTreeRegressor_fill.DecisionTree_imputer(df20),
              RandomForestRegressor_fill.RandomForest_imputer(df20), SVR_fill.SVM_imputer(df20), XGBRegressor_fill.XGBRegressor_imputer(df20), CatBoostRegressor_fill.CatBoostRegressor_imputer(df20)]
df20_result = results_metrics.generate_results_metrics(data, all_methods)
df20_result

In [ ]:
results_metrics_visualization.visualization_results_metrics(df1_result, df15_result, df20_result)